In [1]:
import os
import sys
import pandas as pd

sys.path.append(os.path.abspath('../scripts'))
cleaned_data = pd.read_csv(r'../data/cleaned_data.csv')
cleaned_data.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21304\250740220.py:7: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  cleaned_data = pd.read_csv(r'../data/cleaned_data.csv')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,PCA1,PCA2
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,3.206791,-2.099833
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,3.206791,-2.099833
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,3.190865,-2.112913
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,3.770447,-1.650610
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,3.397998,-1.956487


In [4]:
import numpy as np

cleaned_data['TransactionMonth'] = pd.to_datetime(cleaned_data['TransactionMonth'], errors='coerce')

print(cleaned_data[['mmcode', 'cubiccapacity', 'Cylinders']].describe())

print(cleaned_data[['TotalPremium', 'TotalClaims']].var())

cleaned_data['TotalPremium'] = pd.to_numeric(cleaned_data['TotalPremium'], errors='coerce')
cleaned_data['TotalClaims']=pd.to_numeric(cleaned_data['TotalClaims'], errors='coerce')

cleaned_data.replace([np.inf, -np.inf], np.nan, inplace=True)
cleaned_data['TotalPremium'] = cleaned_data['TotalPremium'].fillna(cleaned_data['TotalPremium'].median())
cleaned_data['TotalClaims'] = cleaned_data['TotalClaims'].fillna(cleaned_data['TotalClaims'].median())

             mmcode  cubiccapacity     Cylinders
count  1.000098e+06   1.000098e+06  1.000098e+06
mean   5.488056e+07   2.466869e+03  4.046616e+00
std    1.360059e+07   4.427106e+02  2.939410e-01
min    4.041200e+06   0.000000e+00  0.000000e+00
25%    6.005692e+07   2.237000e+03  4.000000e+00
50%    6.005842e+07   2.694000e+03  4.000000e+00
75%    6.005842e+07   2.694000e+03  4.000000e+00
max    6.506535e+07   1.288000e+04  1.000000e+01
TotalPremium    5.303096e+04
TotalClaims     5.683812e+06
dtype: float64


In [10]:
from scipy.stats import chi2_contingency, ttest_ind

def select_kpi(df, kpi_column):
    """
    Select the KPI (key performance indicator) column for analysis.

    """
    if kpi_column in df.columns:
        return df[kpi_column]
    else:
        raise ValueError(f"KPI column {kpi_column} not found in the dataset.")
kpi_column = 'TotalPremium'
kpi_data = select_kpi(cleaned_data, kpi_column)

def create_ab_groups(df, feature_column, value_a, value_b):
    """
    Split the dataset into two groups (control and test) based on the feature.
    
    """
    group_a = df[df[feature_column] == value_a]
    group_b = df[df[feature_column] == value_b]
    
    return group_a, group_b
# Step 2: Create Group A (Gauteng) and Group B (KwaZulu-Natal) based on 'Province'
group_a, group_b = create_ab_groups(cleaned_data, feature_column='Province', value_a='Gauteng', value_b='KwaZulu-Natal')

print(f"Group A (Gauteng) Size: {len(group_a)}")
print(f"Group B (KwaZulu-Natal) Size: {len(group_b)}")

def t_test(group_a, group_b, kpi_column):
    """
    Perform a t-test to check if there is a significant difference between two groups for a numerical KPI.
    """
    stat, p = ttest_ind(group_a[kpi_column], group_b[kpi_column], equal_var=False)
    return p
p_value = t_test(group_a, group_b, kpi_column)
print(f"P-value: {p_value}")

def analyze_p_value(p_value, alpha=0.05):
    """
    Analyze the p-value and determine whether to reject the null hypothesis.
    """
    if p_value < alpha:
        return "Reject the null hypothesis. There is a statistically significant difference."
    else:
        return "Fail to reject the null hypothesis. There is no statistically significant difference."

result = analyze_p_value(p_value)
print(result), analyze_p_value

Group A (Gauteng) Size: 393865
Group B (KwaZulu-Natal) Size: 169781
P-value: 1.102783357211804e-210
Reject the null hypothesis. There is a statistically significant difference.


(None, <function __main__.analyze_p_value(p_value, alpha=0.05)>)

In [11]:
import os
import sys
sys.path.append(os.path.abspath('../scripts'))
from chi_squared_test import chi_squared_test

# Example: Performing a chi-squared test on Province
group_a, group_b = create_ab_groups(cleaned_data, feature_column='Province', value_a='Gauteng', value_b='KwaZulu-Natal')
print(f"Group A (Gauteng) Size: {len(group_a)}")
print(f"Group B (KwaZulu-Natal) Size: {len(group_b)}")


# Perform the chi-squared test on a categorical feature (e.g., 'RiskCategory')
p_value = chi_squared_test(group_a, group_b, feature_column='CoverType')
print(f"P-value: {p_value}")


# Analyze the result
result = analyze_p_value(p_value)
print(f"Chi-squared test result: {result}")

Group A (Gauteng) Size: 393865
Group B (KwaZulu-Natal) Size: 169781
P-value: 0.4982636148845796
Chi-squared test result: Fail to reject the null hypothesis. There is no statistically significant difference.


In [12]:
kpi_column = 'TotalClaims'
kpi_data = select_kpi(cleaned_data, kpi_column)


group_a, group_b = create_ab_groups(cleaned_data, feature_column='Province', value_a='Gauteng', value_b='KwaZulu-Natal')
print(f"Group A (Gauteng) Size: {len(group_a)}")
print(f"Group B (KwaZulu-Natal) Size: {len(group_b)}")

p_value = t_test(group_a, group_b, kpi_column)
print(f"P-value: {p_value}")


result = analyze_p_value(p_value)
print(f"Hypothesis 1 - Risk Differences Across Provinces: {result}")

Group A (Gauteng) Size: 393865
Group B (KwaZulu-Natal) Size: 169781
P-value: 0.21269135333580677
Hypothesis 1 - Risk Differences Across Provinces: Fail to reject the null hypothesis. There is no statistically significant difference.


In [13]:
from anova_test import anova_test_for_postalcode_risk

f_stat, p_value = anova_test_for_postalcode_risk(cleaned_data, 'TotalClaims', 'PostalCode')


print(f"F-statistic: {f_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are significant risk differences between zip codes.")
else:
    print("Fail to reject the null hypothesis: There are no significant risk differences between zip codes.")

F-statistic: 0.9419762214391849
P-value: 0.8906511279164051
Fail to reject the null hypothesis: There are no significant risk differences between zip codes.


In [14]:
from anova_test_margin import anova_test_for_postalcode_margin

cleaned_data['Margin'] = cleaned_data['TotalPremium'] - cleaned_data['TotalClaims']

f_stat, p_value = anova_test_for_postalcode_margin(cleaned_data, 'Margin', 'PostalCode')

print(f"F-statistic: {f_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are significant margin (profit) differences between zip codes.")
else:
    print("Fail to reject the null hypothesis: There are no significant margin (profit) differences between zip codes.")

[1459 1513 1619 1625 1629 1852 1982 2007 2066 4093 2000 1577 1610 2410
 6200  122 1520 1709 1739 4000 4066 4091 4342 4359 7784  970 6213 6390
 1868 4310  299  309  152  181 1821 4449 4037  139 4074 1057 7100 9725
 1863 1875 2001 2091 3170 3950 1021 2380  300  302  458 7750  157 4811
 4930 5000 5090 5160 5219 5410 5920 6025 6139 5040 6201 6212 6231 9744
    1    8   64   84  162  164 8000  182  183  186  190 5326  192  194
  199  200  201  208  258  264 1431 1441 1455 1494 1496  284 9762 1507
 1540 1559 1571 1724 1754 1757 1759 1779 1803 1804 1806 1809 1818 1828
 1830 1862 1864 1865 1984 2014 2019 2021 2040 2090 2188 2198 3180 3200
 3245 3310 3380 3609 3610 3612 3613 3370 3600 3629 3630 3650 3780 3900
 3934 3973 5143 3880 3882 3915 4001 4004 4011 4023 4027 4051 4052 4053
 4056 4057 4059 4060 4061 4063 4071 4089 4092 4105 4110 4111 4126 4137
 4140 4180 4200 4240 4260 4309 4340 4360 4450 4700  530  555  556  607
  699  738  827 1022 1123 1133 2375 2415 2499 4068 2951 2952 3602 4182
 4490 

In [15]:
from t_test_men_women import t_test_men_women

cleaned_data['TotalClaims'] = pd.to_numeric(cleaned_data['TotalClaims'], errors='coerce')

group_male = cleaned_data[cleaned_data['Gender'] == 'Male']
group_female = cleaned_data[cleaned_data['Gender'] == 'Female']
t_stat, p_value = t_test_men_women(group_male, group_female, 'TotalClaims')

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are significant risk differences between men and women.")
else:
    print("Fail to reject the null hypothesis: There are no significant risk differences between men and women.")

T-statistic: -0.296353891400699
P-value: 0.7669656471629474
Fail to reject the null hypothesis: There are no significant risk differences between men and women.


In [16]:
from feature_engineering import feature_engineering

df_with_new_features = feature_engineering(cleaned_data)

# Check the new features
print(df_with_new_features[['VehicleAge', 'ClaimsToPremiumRatio', 'VehiclePowerIndex', 'InsuranceTenureMonths', 'IsHighRiskVehicle', 'IsHighRiskRegion']].head())

   VehicleAge  ClaimsToPremiumRatio  VehiclePowerIndex  InsuranceTenureMonths  \
0          20                   0.0       56268.239553             119.766667   
1          20                   0.0       56268.239553             117.733333   
2          20                   0.0       56268.239553             115.700000   
3          20                   0.0       56268.239553             117.733333   
4          20                   0.0       56268.239553             115.700000   

   IsHighRiskVehicle  IsHighRiskRegion  
0                  0                 0  
1                  0                 0  
2                  0                 0  
3                  0                 0  
4                  0                 0  


In [17]:
from encode_categorical_data import encode_categorical_data

# Call the function to encode the categorical data
encoded_data = encode_categorical_data(cleaned_data)

# Check the first few rows after encoding
print(encoded_data.head())

   UnderwrittenCoverID  PolicyID TransactionMonth  IsVATRegistered  \
0               145249     12827       2015-03-01             True   
1               145249     12827       2015-05-01             True   
2               145249     12827       2015-07-01             True   
3               145255     12827       2015-05-01             True   
4               145255     12827       2015-07-01             True   

   MaritalStatus  Gender  PostalCode      mmcode  VehicleType  \
0              1       2        1459  44069150.0            4   
1              1       2        1459  44069150.0            4   
2              1       2        1459  44069150.0            4   
3              1       2        1459  44069150.0            4   
4              1       2        1459  44069150.0            4   

   RegistrationYear  ...  Product_Mobility Metered Taxis: Monthly  \
0              2004  ...                                     True   
1              2004  ...                          

In [18]:
from train_test_data import  train_test_split_data,clean_data_for_modeling,convert_datetime_to_numeric

cleaned_data2=encoded_data['TransactionYear'] = pd.to_datetime(encoded_data['TransactionMonth']).dt.year
cleaned_data2=encoded_data['TransactionMonthNum'] = pd.to_datetime(encoded_data['TransactionMonth']).dt.month

# Drop the original datetime column
encoded_data2 = encoded_data.drop(columns=['TransactionMonth'])
cleaned_data2 = clean_data_for_modeling(encoded_data)
cleaned_data2 = clean_data_for_modeling(encoded_data)


encoded_data2 = convert_datetime_to_numeric(encoded_data)
# Assuming your target column is 'TotalPremium' (or another column you're predicting)
target_column = 'TotalPremium'

# Perform train-test split (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split_data(cleaned_data2, target_column, test_size=0.3)

# Check the shapes to make sure the split is done correctly
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

c:\Users\Administrator\Desktop\KAIM\AlphaCare-Insurance-Solutions\scripts\train_test_data.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
c:\Users\Administrator\Desktop\KAIM\AlphaCare-Insurance-Solutions\scripts\train_test_data.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whic

X_train shape: (700068, 685), X_test shape: (300030, 685)
y_train shape: (700068,), y_test shape: (300030,)


In [20]:
from statistic_modeling import train_linear_regression, train_decision_tree, train_random_forest, train_xgboost, evaluate_model

# Train Linear Regression
linear_model = train_linear_regression(X_train, y_train)
mse, r2 = evaluate_model(linear_model, X_test, y_test)
print(f"Linear Regression MSE: {mse}, R^2: {r2}")

# Train Decision Tree
tree_model = train_decision_tree(X_train, y_train, max_depth=10)
mse, r2 = evaluate_model(tree_model, X_test, y_test)
print(f"Decision Tree MSE: {mse}, R^2: {r2}")

# Train Random Forest
forest_model = train_random_forest(X_train, y_train, n_estimators=100, max_depth=10)
mse, r2 = evaluate_model(forest_model, X_test, y_test)
print(f"Random Forest MSE: {mse}, R^2: {r2}")

# Train XGBoost
xgb_model = train_xgboost(X_train, y_train)
mse, r2 = evaluate_model(xgb_model, X_test, y_test)

print(f"XGBoost MSE: {mse}, R^2: {r2}")

Linear Regression MSE: 6.60208394665799e-08, R^2: 0.9999999999987551
Decision Tree MSE: 1440.9614996764842, R^2: 0.9728301168060332
Random Forest MSE: 1483.1607771183756, R^2: 0.9720344332022566
XGBoost MSE: 8801.170397268836, R^2: 0.8340505476949385
